In [9]:
# Google (test)
############################################
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# system libraries
import os
import sys
import urllib
import pydub
import speech_recognition as sr
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import random
import time
from datetime import datetime
import requests
import json
import stem.process
from stem import Signal
from stem.control import Controller


def delay(waiting_time=5):
    driver.implicitly_wait(waiting_time)
    


driver = webdriver.Chrome("webdriver/chromedriver.exe")
driver.get("https://www.google.com/recaptcha/api2/demo")

frames = driver.find_elements(By.TAG_NAME, "iframe")
# driver.switch_to.frame(recaptcha_control_frame)
driver.switch_to.frame(frames[0])

delay()



# click on checkbox to activate recaptcha
driver.find_element(By.CLASS_NAME, "recaptcha-checkbox-border").click()




try: 
    # If its required to do the challenge
    driver.switch_to.default_content()
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    driver.switch_to.frame(frames[-1])
    
    # click on audio challenge
    time.sleep(10)
    driver.find_element(By.XPATH, "//*[@id='recaptcha-audio-button']").click()
    # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='recaptcha-audio-button']"))).click()


    # switch to recaptcha audio challenge frame
    driver.switch_to.default_content()
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    driver.switch_to.frame(frames[-1])



    # get the mp3 audio file
    delay()
    src = driver.find_element(By.ID, "audio-source").get_attribute("src")
    print(f"[INFO] Audio src: {src}")

    path_to_mp3 = os.path.normpath(os.path.join(os.getcwd(), "sample.mp3"))
    path_to_wav = os.path.normpath(os.path.join(os.getcwd(), "sample.wav"))

    # download the mp3 audio file from the source
    # urllib.request.urlretrieve(src, filename=path_to_mp3)

    def download(url): 
        name = "sample"
        file_name = str(name) + ".mp3"     
        with open(file_name, "wb") as file:         # open in binary mode 
            response = requests.get(url)            # get request 
            time.sleep(2)
            file.write(response.content)            # write to file 

    download(src) 


    # load downloaded mp3 audio file as .wav
    try:
        sound = pydub.AudioSegment.from_mp3(path_to_mp3)
        sound.export(path_to_wav, format="wav")
        sample_audio = sr.AudioFile(path_to_wav)
    except Exception:
        sys.exit(
            "[ERR] Please run program as administrator or download ffmpeg manually, "
            "https://blog.gregzaal.com/how-to-install-ffmpeg-on-windows/"
        )

    # translate audio to text with google voice recognition
    delay()
    r = sr.Recognizer()
    with sample_audio as source:
        audio = r.record(source)
    key = r.recognize_google(audio)
    print(f"[INFO] Recaptcha Passcode: {key}")

    # key in results and submit
    delay()
    driver.find_element(By.ID, "audio-response").send_keys(key.lower())
    driver.find_element(By.ID, "audio-response").send_keys(Keys.ENTER)
except:
    pass

C:\Users\p.farahani\AppData\Local\Temp\ipykernel_13108\1631686865.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("webdriver/chromedriver.exe")


[INFO] Audio src: https://www.google.com/recaptcha/api2/payload?p=06AIIukzi2a_dk7tlmCzfwYLgH0DWvexmn_HxasNsZMbEoPLp_L_b5IsNVEec4Qpf9PGdKc0nQk0Oh-zTYA1gFLg9ODKSB5g9qgUSE4w5bupQkUFp860Hfm0gWcdKZkCabZmILP1O05nLaF4R-47nhZv3kbgcZ2nXlWGxzH15jxs1F9ElxLtb94K3gRw627ImL46jqN_PRwqJK&k=6Le-wvkSAAAAAPBMRTvw0Q4Muexq9bi0DJwx_mJ-
[INFO] Recaptcha Passcode: you really should consider
